In [1]:
import json
import pandas as pd
from pathlib import Path
import os

def find_max_users(folder_path):
    """폴더 내 모든 JSON 파일을 검사하여 최대 user_num을 찾습니다."""
    max_users = 0
    for json_file in Path(folder_path).glob('*.json'):
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            user_num = data['annotations']['user_num']
            max_users = max(max_users, user_num)
    return max_users

def process_json_file(file_path, max_users):
    """단일 JSON 파일을 처리하여 데이터를 추출합니다."""
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
            
            # 기본 메타데이터 추출
            file_info = {
                'file_name': data['meta_data']['file_name'],
                'format': data['meta_data']['format'],
                'size': data['meta_data']['size'],
                'width_height': str(data['meta_data']['width_height']),
                'environment': data['meta_data']['environment'],
                'frame_rate': data['meta_data']['frame_rate'],
                'total_frames': data['meta_data']['total_frames'],
                'camera_height': data['meta_data']['camera_height'],
                'camera_angle': data['meta_data']['camera_angle'],
                'InteractionType': data['additional_info']['InteractionType'],
                'space_context': data['annotations']['space_context']
            }
            
            # 동작별 duration 합계 계산
            action_durations = {0: 0, 1: 0, 2: 0, 3: 0}
            if 'segmentation' in data['annotations']:
                for segment in data['annotations']['segmentation']:
                    action_type = segment['action_type']
                    
                    # action_type이 4인 경우 0으로 처리
                    if action_type == 4:
                        action_type = 0
                    
                    duration = segment['duration']
                    action_durations[action_type] += duration
            
            file_info.update({
                'action_type_0_duration': action_durations[0],
                'action_type_1_duration': action_durations[1],
                'action_type_2_duration': action_durations[2],
                'action_type_3_duration': action_durations[3]
            })
            
            # 현재 파일의 사용자 수
            current_users = data['annotations']['user_num']
            file_info['user_num'] = current_users
            
            # 모든 가능한 사용자 컬럼을 null로 초기화
            for i in range(max_users):
                file_info[f'user_{i}_age'] = None
                file_info[f'user_{i}_gender'] = None
                file_info[f'user_{i}_disability'] = None
            
            # 실제 사용자 정보 채우기
            if 'target_objects' in data['annotations']:
                for i, user in enumerate(data['annotations']['target_objects']):
                    file_info[f'user_{i}_age'] = user['age']
                    file_info[f'user_{i}_gender'] = user['gender']
                    file_info[f'user_{i}_disability'] = user['disability']
                    
            return file_info
        except KeyError as e:
            print(f"오류: '{file_path}' 파일에서 키 '{e.args[0]}'을 찾을 수 없습니다.")
            return None
        except json.JSONDecodeError:
            print(f"오류: '{file_path}' 파일이 유효한 JSON 형식이 아닙니다.")
            return None

def process_folder(folder_path, output_file='kiosk_analysis.csv'):
    """폴더 내의 모든 JSON 파일을 처리하여 CSV 파일로 변환합니다."""
    folder_path = Path(folder_path)
    
    # 폴더 존재 여부 확인
    if not folder_path.exists():
        print(f"오류: '{folder_path}' 폴더를 찾을 수 없습니다.")
        return None
    
    # JSON 파일 존재 여부 확인
    json_files = list(folder_path.glob('*.json'))
    if not json_files:
        print(f"오류: '{folder_path}' 폴더에 JSON 파일이 없습니다.")
        return None
    
    # 최대 사용자 수 찾기
    max_users = find_max_users(folder_path)
    print(f"발견된 최대 사용자 수: {max_users}명")
    
    # 모든 JSON 파일 처리
    all_data = []
    total_files = len(json_files)
    
    print(f"총 {total_files}개의 JSON 파일 처리 중...")
    
    for i, json_file in enumerate(json_files, 1):
        try:
            file_info = process_json_file(json_file, max_users)
            all_data.append(file_info)
            if i % 100 == 0:  # 진행상황 표시
                print(f"{i}/{total_files} 파일 처리 완료")
        except Exception as e:
            print(f"'{json_file}' 파일 처리 중 오류 발생: {e}")
    
    # DataFrame 생성
    df = pd.DataFrame(all_data)
    
    # 컬럼 순서 정렬
    base_columns = [
        'file_name', 'format', 'size', 'width_height', 'environment',
        'frame_rate', 'total_frames', 'camera_height', 'camera_angle',
        'InteractionType', 'space_context',
        'action_type_0_duration', 'action_type_1_duration',
        'action_type_2_duration', 'action_type_3_duration', 'user_num'
    ]
    
    user_columns = []
    for i in range(max_users):
        user_columns.extend([
            f'user_{i}_age',
            f'user_{i}_gender',
            f'user_{i}_disability'
        ])
    
    all_columns = base_columns + user_columns
    df = df[all_columns]
    
    # CSV 파일로 저장
    output_path = folder_path / output_file
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"\n데이터가 성공적으로 저장되었습니다: {output_path}")
    
    return df

if __name__ == "__main__":
    while True:
        # 사용자로부터 폴더 경로 입력 받기
        folder_path = input("JSON 파일이 있는 폴더 경로를 입력해주세요 (종료: 'q'): ").strip()
        
        # 종료 조건
        if folder_path.lower() == 'q':
            print("프로그램을 종료합니다.")
            break
            
        try:
            df = process_folder(folder_path)
            if df is not None:
                print("\n모든 처리가 성공적으로 완료되었습니다!")
                
                # 추가 처리 여부 확인
                continue_process = input("\n다른 폴더도 처리하시겠습니까? (y/n): ").strip().lower()
                if continue_process != 'y':
                    print("프로그램을 종료합니다.")
                    break
                    
        except Exception as e:
            print(f"\n오류가 발생했습니다: {e}")
            print("올바른 폴더 경로로 다시 시도해주세요.")

발견된 최대 사용자 수: 4명
총 2444개의 JSON 파일 처리 중...
100/2444 파일 처리 완료
200/2444 파일 처리 완료
300/2444 파일 처리 완료
400/2444 파일 처리 완료
500/2444 파일 처리 완료
600/2444 파일 처리 완료
700/2444 파일 처리 완료
800/2444 파일 처리 완료
900/2444 파일 처리 완료
1000/2444 파일 처리 완료
1100/2444 파일 처리 완료
1200/2444 파일 처리 완료
1300/2444 파일 처리 완료
1400/2444 파일 처리 완료
1500/2444 파일 처리 완료
1600/2444 파일 처리 완료
1700/2444 파일 처리 완료
1800/2444 파일 처리 완료
1900/2444 파일 처리 완료
2000/2444 파일 처리 완료
2100/2444 파일 처리 완료
2200/2444 파일 처리 완료
2300/2444 파일 처리 완료
2400/2444 파일 처리 완료

데이터가 성공적으로 저장되었습니다: C:\Users\gadi2\Desktop\Downloads\2.라벨링데이터\06. 복지시설_영상 있음\kiosk_analysis.csv

모든 처리가 성공적으로 완료되었습니다!
프로그램을 종료합니다.
